In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings, re
import numpy as np, pandas as pd, matplotlib.pyplot as plt, statsmodels.api as sm, statsmodels.formula.api as smf, seaborn as sns
from pathlib import Path
from IPython.display import display, Markdown, Image
try:
    import yfinance as yf
    YFINANCE_AVAILABLE = True
except ImportError:
    YFINANCE_AVAILABLE = False
try:
    import pandas_datareader.data as web
    PD_READER_AVAILABLE = True
except ImportError:
    PD_READER_AVAILABLE = False

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'font.size': 14, 'figure.figsize': (12, 8), 'figure.dpi': 150})
np.set_printoptions(suppress=True, linewidth=120, precision=4)
warnings.filterwarnings('ignore', category=UserWarning)
sns.set_style('whitegrid')
pd.options.display.float_format = '{:.4f}'.format

# --- Utility Functions ---
def note(msg, **kwargs): display(Markdown(f"<div class='alert alert-block alert-info'>📝 **Note:** {msg}</div>"))
def sec(title): print(f"\n{100*'='}\n| {title.upper()} |\n{100*'='}")

if not YFINANCE_AVAILABLE: note("The 'yfinance' library is not installed (`pip install yfinance`). Some sections will be skipped.")
if not PD_READER_AVAILABLE: note("The 'pandas_datareader' library is not installed (`pip install pandas-datareader`). Some sections will be skipped.")

<a id='hj-bound'></a>
### The Hansen-Jagannathan Bound

Hansen and Jagannathan (1991) provided a powerful, model-free way to visualize the equity premium puzzle. The fundamental pricing equation $1 = E[m R]$ implies a bound on the volatility of any valid SDF:
$$ \frac{\sigma(m)}{E[m]} \ge \frac{|E[R] - R_f|}{\sigma(R)} $$
The right-hand side is the **Sharpe Ratio** of the asset return. This inequality, known as the **Hansen-Jagannathan bound**, states that the volatility of the SDF must be at least as high as the market's Sharpe ratio. The plot below shows this bound. We then plot the volatility of the CCAPM's SDF for various levels of risk aversion ($\gamma$). The CCAPM can only satisfy the bound (i.e., explain the market's high Sharpe ratio) if risk aversion is at an implausibly high level.

In [ ]:
def plot_hj_bound():
    # Market statistics (annualized)
    market_return = 0.08
    risk_free_rate = 0.02
    market_std = 0.16
    sharpe_ratio = (market_return - risk_free_rate) / market_std
    
    # Consumption growth statistics
    cons_growth_std = 0.02
    
    # Plot HJ Bound
    fig, ax = plt.subplots(figsize=(12, 8))
    mean_m = np.linspace(0.95, 1.05, 100)
    hj_bound = sharpe_ratio * mean_m
    ax.plot(mean_m, hj_bound, 'r-', lw=2, label='Hansen-Jagannathan Bound')
    ax.fill_between(mean_m, hj_bound, 3, color='red', alpha=0.1, label='Admissible Region for SDF')

    # Plot CCAPM SDF for different gamma
    gammas = [1, 2, 5, 10, 30, 50]
    sdf_means = []
    sdf_stds = []
    for gamma in gammas:
        # Approximate E[m] and std(m) from CCAPM formula
        mean_m_ccapm = 1 / (1 + risk_free_rate)
        std_m_ccapm = gamma * cons_growth_std * mean_m_ccapm
        sdf_means.append(mean_m_ccapm)
        sdf_stds.append(std_m_ccapm)
        ax.scatter(mean_m_ccapm, std_m_ccapm, s=100, zorder=5)
        ax.annotate(f'γ={gamma}', (mean_m_ccapm, std_m_ccapm), xytext=(5,5), textcoords='offset points')
    
    ax.set_xlabel('Mean of SDF, E[m]')
    ax.set_ylabel('Std. Dev. of SDF, σ(m)')
    ax.set_title('The Equity Premium Puzzle and the Hansen-Jagannathan Bound', fontsize=16)
    ax.legend()
    ax.set_xlim(0.95, 1.02)
    ax.set_ylim(0, 1.0)
    plt.show()

plot_hj_bound()

# Chapter 9.3: Asset Pricing: The SDF Framework and Factor Models

---

### Table of Contents

1.  [**The Unifying Theory: The Stochastic Discount Factor (SDF)**](#sdf)
    - [Deriving the SDF from Micro-foundations](#sdf-derivation)
2.  [**Model 1: The Consumption-Based CAPM (CCAPM)**](#ccapm)
    - [The Equity Premium Puzzle and the Hansen-Jagannathan Bound](#hj-bound)
3.  [**Model 2: The Capital Asset Pricing Model (CAPM)**](#capm)
4.  [**Model 3: The Fama-French Five-Factor Model**](#fama-french)
5.  [**Testing Asset Pricing Models: Fama-MacBeth Regressions**](#fama-macbeth)
6.  [**Unified Case Study**](#case-study)
    - [Data Collection](#data-collection)
    - [Time-Series Regressions (First Pass)](#first-pass)
    - [Fama-MacBeth Regressions (Second Pass)](#second-pass)
    - [Analysis and Interpretation](#analysis)
7.  [**Exercises**](#exercises)
8.  [**Summary and Key Takeaways**](#summary)

<a id='sdf'></a>
## 1. The Unifying Theory: The Stochastic Discount Factor (SDF)

Why do some assets, on average, deliver higher returns than others? Asset pricing theory provides a framework for answering this fundamental question. The central idea is that an asset's expected return is a reward for its **systematic risk**—the risk that cannot be diversified away. This is the risk that is correlated with broad economic states of the world where an extra dollar of wealth is particularly valuable (i.e., "bad times").

All modern asset pricing models can be unified under the elegant and powerful framework of the **Stochastic Discount Factor (SDF)**. The fundamental pricing equation states that the price of any asset, $p_t$, is the expected discounted value of its future payoff, $x_{t+1}$, where the discounting is done by the SDF, $m_{t+1}$:
$$ p_t = E_t[m_{t+1} x_{t+1}] $$

The SDF, also known as the pricing kernel, is a random variable that reflects the marginal utility of wealth in different future states of the world. It is high in "bad times" (when investors are poor and marginal utility is high) and low in "good times." The equation can be rewritten in terms of gross returns, $R_{i, t+1} = x_{t+1}/p_t$, as:
$$ 1 = E_t[m_{t+1} R_{i, t+1}] $$
This implies that an asset's expected return is determined by its covariance with the SDF: assets that have low returns when the SDF is high (i.e., in bad times) are risky and must offer a higher expected return to compensate investors.

Different asset pricing models are simply different assumptions about the specific form of the SDF, $m_{t+1}$.
- **Consumption-Based CAPM (CCAPM):** Assumes the SDF is determined by the growth rate of aggregate consumption.
- **Capital Asset Pricing Model (CAPM):** Assumes the SDF is a linear function of the return on the market portfolio.
- **Multi-Factor Models:** Assume the SDF is a linear function of several risk factors (e.g., market, size, value).

This notebook traces this intellectual journey, starting with the foundational theoretical models and progressing to the multi-factor workhorses of modern empirical finance.

<a id='sdf-derivation'></a>
### Deriving the SDF from Micro-foundations
The SDF is not an ad-hoc assumption; it arises naturally from the optimization problem of an economic agent. Consider a representative agent choosing consumption ($C_t$) and their holdings of an asset ($z_t$) to maximize lifetime utility:
$$ \max E_0 \sum_{t=0}^\infty \beta^t u(C_t) \quad \text{s.t.} \quad C_t + p_t z_t = (p_t + d_t)z_{t-1} $$
where $p_t$ is the asset price and $d_t$ is its dividend. The first-order condition with respect to asset holdings $z_t$ is:
$$ -p_t u'(C_t) + E_t[\beta u'(C_{t+1})(p_{t+1} + d_{t+1})] = 0 $$
Rearranging this gives the fundamental asset pricing equation:
$$ p_t = E_t \left[ \underbrace{\beta \frac{u'(C_{t+1})}{u'(C_t)}}_{m_{t+1}} (p_{t+1} + d_{t+1}) \right] = E_t[m_{t+1} x_{t+1}] $$
This shows that the Stochastic Discount Factor is the **intertemporal marginal rate of substitution**. It is the rate at which an investor is willing to trade consumption today for consumption tomorrow, and it is this rate that is used to discount future payoffs.

---

<a id='ccapm'></a>
## 2. Model 1: The Consumption-Based CAPM (CCAPM) and its Puzzles

The CCAPM is the most theoretically pure asset pricing model. It arises from specifying a particular functional form for the utility function in the derivation above. For a consumer with a power utility function $u(C) = \frac{C^{1-\gamma}}{1-\gamma}$, the marginal utility is $u'(C) = C^{-\gamma}$. Plugging this into the definition of the SDF gives the famous result:
$$ m_{t+1} = \beta \left( \frac{C_{t+1}}{C_t} \right)^{-\gamma} $$
where $\beta$ is the subjective discount factor and $\gamma$ is the coefficient of relative risk aversion. Substituting this into the fundamental pricing equation yields the CCAPM:
$$ E_t[R_i] - R_f = \text{Cov}_t\left(R_i, -\frac{\Delta C_{t+1}}{C_t}\right) \cdot \gamma $$
The model predicts that an asset's excess return is determined by its covariance with aggregate consumption growth. Assets that pay off when consumption is low (e.g., insurance) are valuable and have low expected returns, while assets that do poorly when consumption is low (e.g., cyclical stocks) are risky and must have high expected returns.

Despite its theoretical elegance, the CCAPM performs poorly empirically, leading to two famous puzzles:
1.  **The Equity Premium Puzzle (Mehra & Prescott, 1985):** Historically, the US equity premium (the return of stocks over bonds) has been around 6-8%. Aggregate consumption growth is very smooth (low variance). To explain the high equity premium with a low consumption variance, the CCAPM requires an implausibly high coefficient of risk aversion ($\gamma > 30$).
2.  **The Risk-Free Rate Puzzle:** With a high $\gamma$, the model predicts a very high risk-free rate, far higher than the ~1-2% observed historically. This is because a high $\gamma$ implies a strong desire for intertemporal substitution, leading agents to save heavily and drive up the price of risk-free assets (driving down their return).

<a id='capm'></a>
## 3. Model 2: The Capital Asset Pricing Model (CAPM)

Frustrated by the empirical failures of the CCAPM, finance academics developed models where the SDF is a function of asset returns themselves, which are more volatile and have a higher correlation with individual stock returns. The CAPM is the most famous example. It assumes the SDF is a linear function of the return on the market portfolio, $R_m$. This gives rise to the classic CAPM equation:
$$ E[R_i] - R_f = \beta_i (E[R_m] - R_f) $$
Where beta, $\beta_i = \frac{\text{Cov}(R_i, R_m)}{\text{Var}(R_m)}$, measures the asset's systematic risk exposure to the market portfolio.

To test the CAPM, we transform it into a regression model by adding an intercept ($\alpha_i$) and an error term ($\epsilon_{it}$), yielding the **Security Characteristic Line**:
$$ (R_{it} - R_{ft}) = \alpha_i + \beta_i (R_{mt} - R_{ft}) + \epsilon_{it} $$
The intercept, **Jensen's alpha**, is the key test. The CAPM predicts $\alpha_i=0$ for all assets. A non-zero alpha indicates that an asset has earned returns that are unexplained by its market risk, representing a model anomaly.

<a id='fama-french'></a>
## 4. Model 3: The Fama-French Five-Factor Model

The persistence of CAPM anomalies led Fama and French to propose that other sources of systematic risk, beyond the market factor, are priced by investors. Their five-factor model is an empirical model where the SDF is assumed to be a linear function of five factors: market, size, value, profitability, and investment.

The regression equation is:
$$ (R_{it} - R_{ft}) = \alpha_i + \beta_{mkt} (R_{mt} - R_{ft}) + \beta_{smb} SMB_t + \beta_{hml} HML_t + \beta_{rmw} RMW_t + \beta_{cma} CMA_t + \epsilon_{it} $$

The factors are constructed as long-short portfolios capturing specific characteristics:
1.  **SMB (Small Minus Big):** Captures the 'size' premium.
2.  **HML (High Minus Low):** Captures the 'value' premium.
3.  **RMW (Robust Minus Weak):** Captures the 'profitability' premium.
4.  **CMA (Conservative Minus Aggressive):** Captures the 'investment' premium.

The goal of this model is to see if these additional factors can explain the portion of returns (the alpha) that the CAPM leaves on the table. A successful model would produce an alpha that is statistically indistinguishable from zero.

<a id='fama-macbeth'></a>
## 5. Testing Asset Pricing Models: Fama-MacBeth Regressions

The previous analysis shows *how* assets load on factors. But it doesn't answer a more fundamental question: do these factors actually carry a risk premium? In other words, does the market reward investors with higher average returns for taking on, for example, value risk (a high HML beta)?

To answer this, we turn to the **Fama-MacBeth (1973) two-step regression** procedure, a cornerstone of empirical asset pricing. It is designed to test whether the risk premium associated with a factor is statistically significant.

**The Two-Step Procedure:**

1.  **First Pass (Time-Series Regression):** For each of the $N$ assets (e.g., stocks or portfolios) in our sample, we run a time-series regression of its excess returns on the proposed risk factors. This gives us a set of factor loadings (betas), $\hat{\beta}_{i,k}$, for each asset $i$ on each factor $k$. These betas measure the sensitivity of the asset's return to the factor's movements.

2.  **Second Pass (Cross-Sectional Regressions):** For *each time period* $t$, we run a single cross-sectional regression of all asset returns at that time $t$ on the betas we estimated in the first pass. The coefficients from this regression, $\hat{\lambda}_{k,t}$, are interpreted as the **risk premium** for factor $k$ at time $t$.

After running the second pass for all $T$ time periods, we have a time series of risk premia estimates for each factor. The average risk premium for factor $k$ is the time-series average of these estimates, $\bar{\lambda}_k$. We use a t-statistic to test if this average premium is statistically different from zero. If it is, we conclude that the factor is "priced".

The diagram below illustrates this two-pass procedure.

![Fama-MacBeth Two-Pass Regression Procedure](images/fama_macbeth_procedure.png)

<a id='case-study'></a>
## 6. Unified Case Study: Testing Asset Pricing Models

<a id='data-collection'></a>
### Data Collection
We begin by fetching the necessary data: the Fama-French 5 factors and the 10 Industry Portfolios, which will serve as our test assets. We will download this directly from Ken French's data library to ensure reproducibility.

In [ ]:
if PD_READER_AVAILABLE:
    note("Attempting to download Fama-French data from Ken French's Data Library...")
    try:
        # Download FF5 Factors
        ff_factors_raw = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench', start='1963-07-01')
        ff_factors = ff_factors_raw[0] / 100 # Monthly data
        ff_factors.index = ff_factors.index.to_timestamp()

        # Download 10 Industry Portfolios
        ind_port_raw = web.DataReader('10_Industry_Portfolios', 'famafrench', start='1963-07-01')
        industry_portfolios = ind_port_raw[0] / 100
        industry_portfolios.index = industry_portfolios.index.to_timestamp()

        note("Data downloaded and prepared successfully.")
        DATA_LOADED_SUCCESS = True

    except Exception as e:
        note(f"Could not download data from Ken French's library ({e}). Please check your internet connection. Case study will be skipped.")
        DATA_LOADED_SUCCESS = False
else:
    note("pandas_datareader is not installed. Case study will be skipped.")
    DATA_LOADED_SUCCESS = False

if DATA_LOADED_SUCCESS:
    ff_factors.rename(columns={'Mkt-RF': 'MKT'}, inplace=True)
    # Calculate excess returns for industry portfolios
    excess_returns = industry_portfolios.subtract(ff_factors['RF'], axis=0)
    
    print("\nFama-French 5 Factors (Head):")
    display(ff_factors.head())
    print("\nExcess Returns of 10 Industry Portfolios (Head):")
    display(excess_returns.head())

<a id='first-pass'></a>
### Time-Series Regressions (First Pass)

First, we estimate the factor loadings (betas) for each of the 10 industry portfolios. We will do this for both the CAPM (which only uses the MKT factor) and the Fama-French 5-Factor model. These betas tell us the sensitivity of each portfolio to the systematic risk factors.

In [ ]:
def get_factor_betas(excess_returns, factors):
    """Runs time-series regressions to get factor betas for each asset."""
    betas = {}
    X = sm.add_constant(factors)
    for portfolio in excess_returns.columns:
        y = excess_returns[portfolio]
        model = sm.OLS(y, X).fit()
        betas[portfolio] = model.params
    return pd.DataFrame(betas).T

if DATA_LOADED_SUCCESS:
    # CAPM Betas
    capm_betas = get_factor_betas(excess_returns, ff_factors[['MKT']])
    
    # Fama-French 5-Factor Betas
    ff5_factors = ff_factors[['MKT', 'SMB', 'HML', 'RMW', 'CMA']]
    ff5_betas = get_factor_betas(excess_returns, ff5_factors)
    
    note("**CAPM Betas (First Pass Time-Series Regression Results)**")
    display(capm_betas)
    
    note("**Fama-French 5-Factor Betas (First Pass Time-Series Regression Results)**")
    display(ff5_betas)

<a id='second-pass'></a>
### Fama-MacBeth Regressions (Second Pass)

Now we proceed to the second pass. For each month in our sample, we run a cross-sectional regression of that month's 10 industry excess returns on their previously estimated betas. The coefficients from these regressions are the monthly risk premia for each factor. We then average these monthly premia over time and use a t-test to see if they are statistically significant.

In [ ]:
def run_fama_macbeth(excess_returns, betas, lags=6):
    """
    Performs the Fama-MacBeth two-pass regression with Newey-West standard error correction.

    Args:
        excess_returns (pd.DataFrame): DataFrame of asset excess returns (T x N).
        betas (pd.DataFrame): DataFrame of factor betas from the first pass (N x K).
        lags (int): Number of lags to use for the Newey-West correction.

    Returns:
        pd.DataFrame: A DataFrame containing the average risk premia, Newey-West t-statistics,
                      and p-values for each factor.
    """
    # Betas from the first pass are the independent variables in the second pass.
    # We do NOT add a constant here, as the intercept from the regression is the zero-beta premium.
    X_betas = betas.copy()

    # --- Second Pass: Cross-sectional regressions for each time period ---
    lambdas = []
    for t in excess_returns.index:
        # For each month t, regress that month's returns on the betas.
        y_t = excess_returns.loc[t]
        X_t = sm.add_constant(X_betas) # Add constant to get the zero-beta premium (lambda_0)
        
        model = sm.OLS(y_t, X_t).fit()
        lambdas.append(model.params)
    
    lambdas_df = pd.DataFrame(lambdas, index=excess_returns.index, columns=['Zero-Beta Alpha'] + list(betas.columns))

    # --- Time-series analysis of the risk premia ---
    avg_lambdas = lambdas_df.mean()
    
    # Calculate standard errors and t-statistics using Newey-West
    t_stats = {}
    p_values = {}
    std_errors = {}
    
    for factor in lambdas_df.columns:
        lambda_series = lambdas_df[factor]
        
        # Regress the time series of lambdas on a constant to get the mean
        # and its HAC standard error. This is the standard Fama-MacBeth procedure.
        model = sm.OLS(lambda_series, sm.add_constant(np.ones(len(lambda_series)))).fit(
            cov_type='HAC', cov_kwds={'maxlags': lags}
        )
        
        t_stats[factor] = model.tvalues[0]
        p_values[factor] = model.pvalues[0]
        std_errors[factor] = model.bse[0]

    results = pd.DataFrame({
        'Risk Premium (%/month)': avg_lambdas * 100,
        'Std. Error (NW)': std_errors,
        'T-statistic (NW)': pd.Series(t_stats),
        'P-value (NW)': pd.Series(p_values)
    }).round(4)
    results.index.name = 'Factor'
    
    return results

if DATA_LOADED_SUCCESS:
    # Fama-MacBeth for CAPM
    capm_fm_results = run_fama_macbeth(excess_returns, capm_betas, lags=6)
    note("**Fama-MacBeth Results for CAPM**")
    display(capm_fm_results.style.format('{:.4f}').applymap(lambda v: 'font-weight: bold;' if abs(v) > 1.96 else '', subset=['T-statistic (NW)']))

    # Fama-MacBeth for FF5
    ff5_fm_results = run_fama_macbeth(excess_returns, ff5_betas, lags=6)
    note("**Fama-MacBeth Results for Fama-French 5-Factor Model**")
    display(ff5_fm_results.style.format('{:.4f}').applymap(lambda v: 'font-weight: bold;' if abs(v) > 1.96 else '', subset=['T-statistic (NW)']))

<a id='analysis'></a>
### Analysis and Interpretation

The results from the Fama-MacBeth regressions, now with robust Newey-West standard errors, allow us to test which factors command a statistically significant risk premium. We look at the **T-statistic (NW)** and **P-value (NW)** columns. A factor is considered "priced" if its p-value is below a conventional significance level (e.g., 0.05), which corresponds to a t-statistic with an absolute value greater than approximately 1.96.

1.  **CAPM Results:**
    - The table shows the risk premium for the market factor (MKT). By examining its t-statistic and p-value, we can determine if systematic market risk is rewarded.
    - The intercept (`const`) represents the average excess return of a portfolio with zero beta. According to the CAPM, this should be zero. The t-statistic tells us if the observed intercept is statistically distinguishable from zero.

2.  **Fama-French 5-Factor Results:** The multi-factor model provides a more nuanced picture:
    - We again check the **MKT** premium. Does it remain significant after controlling for the other factors?
    - **HML (Value):** A significant positive premium would confirm the "value effect"—that the market rewards investors for holding high book-to-market (value) stocks.
    - **RMW (Profitability):** A significant premium indicates that profitability is a priced risk factor.
    - **SMB (Size)** and **CMA (Investment):** We examine the significance of these factors to see if size and investment patterns are rewarded with higher average returns.

**Conclusion:** By comparing the results of the two models, we can assess whether the Fama-French 5-factor model provides a better description of the cross-section of returns than the single-factor CAPM. The factors with statistically significant t-statistics are the ones that are robustly priced sources of systematic risk over our sample period.

<a id='exercises'></a>
## 7. Exercises

1.  **The SDF Framework:** Explain the Stochastic Discount Factor (SDF) equation $1 = E[m R_i]$. What is the economic intuition of the SDF, $m$? How can this single equation give rise to both the CCAPM and the Fama-French models under different assumptions about what $m$ is?

2.  **Equity Premium Puzzle:** In your own words, what is the Equity Premium Puzzle? Why can't the standard CCAPM, with a reasonable level of risk aversion (e.g., $\gamma < 10$), explain the historically high excess return of the stock market? What does this imply about the covariance between consumption growth and stock market returns?

3.  **Alternative Test Assets:** Rerun the Fama-MacBeth analysis in the case study, but instead of the 10 Industry Portfolios, use the `25_Portfolios_5x5` (portfolios sorted on size and book-to-market) as the test assets. Do the conclusions about which factors are priced change? Why might using these portfolios provide a more powerful test?

4.  **Interpreting Alpha:** In the 5-factor model time-series regressions (the first pass), if a portfolio still had a significant positive alpha, what would be the possible interpretations? Does it necessarily imply market inefficiency, or could it point to missing risk factors not captured by the Fama-French 5? What kind of risk might be missing (e.g., momentum, liquidity)?
5.  **Arbitrage Pricing Theory (APT):** Research the Arbitrage Pricing Theory (APT) proposed by Stephen Ross. How does its derivation of a multi-factor model differ from the equilibrium-based approach of the CAPM? Which set of assumptions do you find more plausible?

<a id='summary'></a>
## 8. Summary and Key Takeaways

This notebook provided a tour of modern empirical asset pricing, unified by the concept of the Stochastic Discount Factor. We journeyed from the elegant but empirically challenged CCAPM to the workhorse factor models that dominate empirical research today.

**Key Takeaways:**
- The **SDF equation** ($p_t = E_t[m_{t+1} x_{t+1}]$) is the unifying principle of all asset pricing.
- Different models are just different specifications of the SDF, $m_{t+1}$.
- The **CCAPM**, while theoretically pure, fails to match the data due to the smoothness of consumption growth (the Equity Premium Puzzle).
- The **CAPM** simplifies the SDF to be a linear function of the market return, providing a better, though incomplete, description of risk.
- **Multi-factor models**, like the Fama-French 5-Factor model, capture more dimensions of systematic risk (size, value, profitability, investment) and provide a much better explanation of the cross-section of stock returns.
- The **Fama-MacBeth regression** is the standard tool for testing whether these systematic risk factors carry a statistically significant risk premium. Our analysis showed that market, value, and profitability risk have been historically rewarded.